### Fourth exercise: Match searches with bookings

The goal of this exercise is to add a match column to the searches file to indicate whether the search ended up in a booking 1 or no 0, then write a csv file with searches file with the match column

In [1]:
import os
import pandas as pd

In [2]:
# Select the fields to work with for that problem
fields_searches = ['Date', 'Time','Origin', 'Destination', 'Seg1Date']
fields_bookings = ['cre_date','dep_port', 'arr_port', 'pax', 'brd_time']

In [3]:
# Import the searches and bookings file
searches = pd.read_csv('/users/driss/Downloads/test_files/searches.csv', sep = "^", usecols=fields_searches, skipinitialspace=True)
bookings =  pd.read_csv('/users/driss/Downloads/test_files/bookings.csv', sep = "^", usecols=fields_bookings, skipinitialspace=True)

In [4]:
# Remove the bookings that have been canceled
bookings = bookings[bookings['pax'] > 0]

In [5]:
# Define the Time fucntion to separate the date from the time in the dataframe
def Time(row):
    cre_date = row['cre_date']
    time = cre_date[10:]
    return time

In [6]:
bookings['Time'] = bookings.apply(lambda row : Time(row), axis = 1)

In [7]:
# Define the function Date to keep only the date in the cre_date column
def Date(row):
    cre_date = row['cre_date']
    date = cre_date[0:10]
    return date

In [8]:
bookings['cre_date'] = bookings.apply(lambda row : Date(row), axis = 1)

In [9]:
# Define the function brd to keep only the date in the brd_time column
def brd(row):
    brd_time = row['brd_time']
    date = brd_time[0:10]
    return date

In [10]:
bookings['brd_time'] = bookings.apply(lambda row : brd(row), axis = 1)

In [11]:
a = searches[:10]
b = bookings[:10]

In [12]:
a

,Date,Time,Origin,Destination,Seg1Date
0,2013-01-01,20:25:57,TXL,AUH,2013-01-26
1,2013-01-01,10:15:33,ATH,MIL,2013-01-04
2,2013-01-01,18:04:49,ICT,SFO,2013-08-02
3,2013-01-01,17:42:40,RNB,ARN,2013-01-02
4,2013-01-01,17:48:29,OSL,MAD,2013-03-22
5,2013-01-01,22:00:28,IAH,BLR,2013-02-14
6,2013-01-01,10:47:14,CPH,PAR,2013-03-02
7,2013-01-01,23:39:49,PAR,DUB,2013-02-12
8,2013-01-01,17:08:46,DUS,ACE,2013-03-30
9,2013-01-01,19:57:57,FRA,BGW,2013-02-26


In [13]:
b

,cre_date,dep_port,arr_port,brd_time,pax,Time
1,2013-03-26,SAL,CLT,2013-04-12,1.0,00:00:00
2,2013-03-26,SAL,CLT,2013-07-15,1.0,00:00:00
3,2013-03-26,AKL,SVO,2013-04-24,1.0,00:00:00
4,2013-03-26,AKL,SVO,2013-05-14,1.0,00:00:00
5,2013-03-20,DEN,LGA,2013-04-07,1.0,00:00:00
6,2013-03-20,DEN,LGA,2013-04-11,1.0,00:00:00
7,2013-03-25,NRT,SIN,2013-04-14,2.0,00:00:00
8,2013-03-25,NRT,SIN,2013-04-16,2.0,00:00:00
9,2013-03-25,NRT,SIN,2013-04-18,2.0,00:00:00
10,2013-03-25,NRT,SIN,2013-04-19,2.0,00:00:00


In [25]:
# function to check if the match occured for a search that happened before a booking
import math
def check(row):
    if(math.isnan(row['match'])):
        return 0
    elif(row['Date'] > row['cre_date']):
        return 0
    else:
        return 1

In [15]:
# Remove the duplicates from the bookings file
bookingsWithoutDuplicates = bookings.drop_duplicates()

In [16]:
# Creating a column match with value 1 for all observations
bookingsWithoutDuplicates['match'] = 1

/Users/driss/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [17]:
bookingsWithoutDuplicates[:10]

,cre_date,dep_port,arr_port,brd_time,pax,Time,match
1,2013-03-26,SAL,CLT,2013-04-12,1.0,00:00:00,1
2,2013-03-26,SAL,CLT,2013-07-15,1.0,00:00:00,1
3,2013-03-26,AKL,SVO,2013-04-24,1.0,00:00:00,1
4,2013-03-26,AKL,SVO,2013-05-14,1.0,00:00:00,1
5,2013-03-20,DEN,LGA,2013-04-07,1.0,00:00:00,1
6,2013-03-20,DEN,LGA,2013-04-11,1.0,00:00:00,1
7,2013-03-25,NRT,SIN,2013-04-14,2.0,00:00:00,1
8,2013-03-25,NRT,SIN,2013-04-16,2.0,00:00:00,1
9,2013-03-25,NRT,SIN,2013-04-18,2.0,00:00:00,1
10,2013-03-25,NRT,SIN,2013-04-19,2.0,00:00:00,1


In [18]:
# Rename the attributes of the bookings file in order to compute a merge with the searches file
bookingsWithoutDuplicates.rename(columns={'brd_time':'Seg1Date','dep_port': 'Origin', 'arr_port':'Destination'}, inplace=True)

/Users/driss/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/frame.py:2748: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [19]:
# Merge the searches file with the bookings file on the date of the flight, the departure and arrival airports
searches_merged = pd.merge(searches, bookingsWithoutDuplicates, on = ['Seg1Date', 'Origin', 'Destination'], how = 'left')

In [20]:
a = searches_merged[0:20]

In [21]:
a

,Date,Time_x,Origin,Destination,Seg1Date,cre_date,pax,Time_y,match
0,2013-01-01,20:25:57,TXL,AUH,2013-01-26,NaN,NaN,NaN,NaN
1,2013-01-01,10:15:33,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN
2,2013-01-01,18:04:49,ICT,SFO,2013-08-02,NaN,NaN,NaN,NaN
3,2013-01-01,17:42:40,RNB,ARN,2013-01-02,NaN,NaN,NaN,NaN
4,2013-01-01,17:48:29,OSL,MAD,2013-03-22,2013-01-14,1.0,00:00:00,1.0
5,2013-01-01,22:00:28,IAH,BLR,2013-02-14,NaN,NaN,NaN,NaN
6,2013-01-01,10:47:14,CPH,PAR,2013-03-02,NaN,NaN,NaN,NaN
7,2013-01-01,23:39:49,PAR,DUB,2013-02-12,NaN,NaN,NaN,NaN
8,2013-01-01,17:08:46,DUS,ACE,2013-03-30,NaN,NaN,NaN,NaN
9,2013-01-01,19:57:57,FRA,BGW,2013-02-26,NaN,NaN,NaN,NaN


In [26]:
a['match'] = a.apply(lambda row : check(row), axis = 1)

/Users/driss/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
a

,Date,Time_x,Origin,Destination,Seg1Date,cre_date,pax,Time_y,match
0,2013-01-01,20:25:57,TXL,AUH,2013-01-26,NaN,NaN,NaN,0
1,2013-01-01,10:15:33,ATH,MIL,2013-01-04,NaN,NaN,NaN,0
2,2013-01-01,18:04:49,ICT,SFO,2013-08-02,NaN,NaN,NaN,0
3,2013-01-01,17:42:40,RNB,ARN,2013-01-02,NaN,NaN,NaN,0
4,2013-01-01,17:48:29,OSL,MAD,2013-03-22,2013-01-14,1.0,00:00:00,1
5,2013-01-01,22:00:28,IAH,BLR,2013-02-14,NaN,NaN,NaN,0
6,2013-01-01,10:47:14,CPH,PAR,2013-03-02,NaN,NaN,NaN,0
7,2013-01-01,23:39:49,PAR,DUB,2013-02-12,NaN,NaN,NaN,0
8,2013-01-01,17:08:46,DUS,ACE,2013-03-30,NaN,NaN,NaN,0
9,2013-01-01,19:57:57,FRA,BGW,2013-02-26,NaN,NaN,NaN,0


In [28]:
# check if the match occured for a search that happened before a booking (if not match takes value 0)
searches_merged['match'] = searches_merged.apply(lambda row : check(row), axis = 1)

In [38]:
# Replace NAN values that correspond to a no match in the match column with 0
#searches_merged['match'] = searches_merged['match'].fillna(0)

In [31]:
searches_merged[:20]

,Date,Time_x,Origin,Destination,Seg1Date,cre_date,pax,Time_y,match
0,2013-01-01,20:25:57,TXL,AUH,2013-01-26,NaN,NaN,NaN,0
1,2013-01-01,10:15:33,ATH,MIL,2013-01-04,NaN,NaN,NaN,0
2,2013-01-01,18:04:49,ICT,SFO,2013-08-02,NaN,NaN,NaN,0
3,2013-01-01,17:42:40,RNB,ARN,2013-01-02,NaN,NaN,NaN,0
4,2013-01-01,17:48:29,OSL,MAD,2013-03-22,2013-01-14,1.0,00:00:00,1
5,2013-01-01,22:00:28,IAH,BLR,2013-02-14,NaN,NaN,NaN,0
6,2013-01-01,10:47:14,CPH,PAR,2013-03-02,NaN,NaN,NaN,0
7,2013-01-01,23:39:49,PAR,DUB,2013-02-12,NaN,NaN,NaN,0
8,2013-01-01,17:08:46,DUS,ACE,2013-03-30,NaN,NaN,NaN,0
9,2013-01-01,19:57:57,FRA,BGW,2013-02-26,NaN,NaN,NaN,0


In [32]:
# Delete fields not related to the searches file
del searches_merged['pax']
del searches_merged['Time_y']
del searches_merged['cre_date']

In [33]:
searches_merged[:40]

,Date,Time_x,Origin,Destination,Seg1Date,match
0,2013-01-01,20:25:57,TXL,AUH,2013-01-26,0
1,2013-01-01,10:15:33,ATH,MIL,2013-01-04,0
2,2013-01-01,18:04:49,ICT,SFO,2013-08-02,0
3,2013-01-01,17:42:40,RNB,ARN,2013-01-02,0
4,2013-01-01,17:48:29,OSL,MAD,2013-03-22,1
5,2013-01-01,22:00:28,IAH,BLR,2013-02-14,0
6,2013-01-01,10:47:14,CPH,PAR,2013-03-02,0
7,2013-01-01,23:39:49,PAR,DUB,2013-02-12,0
8,2013-01-01,17:08:46,DUS,ACE,2013-03-30,0
9,2013-01-01,19:57:57,FRA,BGW,2013-02-26,0


In [ ]:
# Write the searches file into a csv file with a match attribute to indicate if a search ended up in a booking
searches_merged.to_csv("/users/driss/Downloads/Test/searches_with_match.csv", sep = "^")